In [103]:
import loginkey

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

In [52]:
class element_not_have_css_class(object):
    """An expectation for checking that an element has a particular css class.

    locator - used to find the element
    returns the WebElement once it has the particular css class
    """
    def __init__(self, locator, css_class):
        self.locator = locator
        self.css_class = css_class

    def __call__(self, driver):
        element = driver.find_element(*self.locator)   # Finding the referenced element
        if self.css_class not in element.get_attribute("class"):
            return element
        else:
            return False

In [106]:
driver = webdriver.Chrome()
driver.get('https://www.artsy.net/')

In [109]:
button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div/header/div/nav/div[1]/a[2]')

# user = loginkey.user
# password = loginkey.password

user = 'michaeljroth815@gmail.com'
password = 'YudBYiLj$e7u2Xq'

driver.find_element(By.CLASS_NAME, 'hgOWHr').click()
driver.find_element(By.XPATH, "//input[@name='email']").send_keys(user)
driver.find_element(By.CLASS_NAME, 'jGXstL').click()
driver.find_element(By.XPATH, "//input[@name='password']").send_keys(password)
driver.find_element(By.XPATH, "//button[@type='submit']").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x0000000100805170 chromedriver + 4477296
1   chromedriver                        0x000000010079cc18 chromedriver + 4049944
2   chromedriver                        0x00000001003eec10 chromedriver + 191504
3   chromedriver                        0x000000010041e988 chromedriver + 387464
4   chromedriver                        0x0000000100414868 chromedriver + 346216
5   chromedriver                        0x0000000100414254 chromedriver + 344660
6   chromedriver                        0x000000010044685c chromedriver + 551004
7   chromedriver                        0x0000000100412f58 chromedriver + 339800
8   chromedriver                        0x00000001007dabd4 chromedriver + 4303828
9   chromedriver                        0x00000001007df094 chromedriver + 4321428
10  chromedriver                        0x00000001007e3600 chromedr

In [ ]:
auction_results = []



In [27]:
artist = 'gustav-klimt'

url = f'https://www.artsy.net/artist/{artist}/auction-results?metric=in'

driver.get(url)

In [71]:

select_element = driver.find_elements(By.XPATH,'//select')
select_object = Select(select_element[1])
select_object.select_by_index(1)

In [75]:
driver.switch_to.window(driver.window_handles[0])

soup = driver.page_source

driver.find_element(By.LINK_TEXT, 'Next').click()

element = wait.until(element_not_have_css_class((By.CLASS_NAME, 'beISLe'), "loading"))

In [110]:
driver.close()